여성 의류 리뷰 데이터셋을 이용한 감정 분석

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

데이터셋을 읽어옵니다.

In [ ]:
train_data = pd.read_csv("/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv")
train_data.head()

In [ ]:
train_data.nunique()

In [ ]:
train_data = train_data[~train_data['Review Text'].isnull()]
train_data.shape

리뷰 테이터 중 null값이 있는 요소를 확인합니다.

In [ ]:
print(train_data['Review Text'])

In [ ]:
train_data.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
train_data.drop("Clothing ID",axis=1,inplace=True)

In [ ]:
train_data.drop("Age",axis=1,inplace=True)

In [ ]:
train_data.drop("Title",axis=1,inplace=True)

In [ ]:
train_data.drop("Positive Feedback Count",axis=1,inplace=True)

In [ ]:
train_data.drop("Division Name",axis=1,inplace=True)

In [ ]:
train_data.drop("Department Name",axis=1,inplace=True)

In [ ]:
train_data.drop("Class Name",axis=1,inplace=True)

필요없는 항목들을 삭제해줍니다.

In [ ]:
train_data.head()

In [ ]:
train_data['Review Text'].nunique(),train_data['Recommended IND'].nunique()

In [ ]:
train_data.drop_duplicates(subset=['Review Text'], inplace=True)

In [ ]:
print('총 샘플의 수 :',len(train_data))

In [ ]:
train_data['Recommended IND'].value_counts().plot(kind = 'bar')

In [ ]:
print(train_data.groupby('Recommended IND').size().reset_index(name = 'count'))

In [ ]:
print(train_data.isnull().values.any())

In [ ]:
import re
text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
re.sub(r'[^a-zA-Z ]', '', text) #알파벳과 공백을 제외하고 모두 제거

In [ ]:
train_data['Review Text'] = train_data['Review Text'].str.replace("[^a-zA-Z ]","")
# 글과 공백을 제외하고 모두 제거
train_data[:5]

리뷰를 토큰화 하기 전에 글과 공백을 제외한 다른 요소들을 제거합니다.

In [ ]:
train_data['Review Text'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer=TreebankWordTokenizer()
text=train_data['Review Text'][0]
print(tokenizer.tokenize(text))

첫 번째 리뷰를 토큰화하여 확인

In [ ]:
X_train = []
for sentence in train_data['Review Text']:
    temp_X = []
    tokenizer=TreebankWordTokenizer()
    temp_X=sentence
    temp_X=tokenizer.tokenize(temp_X)
    X_train.append(temp_X)

In [ ]:
print(X_train[:3])

전체 리뷰를 토큰화한 후 앞에 3개만 확인해봅니다.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(X_train)

In [ ]:
print(tokenizer2.word_index)

토큰화된 단어들에 인덱스를 부여합니다.

In [ ]:
threshold = 3
total_cnt = len(tokenizer2.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer2.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer2 = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer2.fit_on_texts(X_train)
X_train = tokenizer2.texts_to_sequences(X_train)

In [ ]:
print(X_train[:3])

In [ ]:
y_train = np.array(train_data['Recommended IND'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
print(len(X_train))


In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
import matplotlib.pyplot as plt
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 100
below_threshold_len(max_len, X_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, maxlen = max_len)

리뷰의 길이를 100으로 맞춰 정규화 시켜줍니다.

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
print(len(X_train))

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

LSTM

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_train, y_train)[1]))

In [ ]:
def sentiment_predict(new_sentence):
    tokenizer.tokenize(new_sentence) # 토큰화
    encoded = tokenizer2.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('i love this dress its soo pretty')

In [ ]:
sentiment_predict('Its too dark.')

In [ ]:
sentiment_predict('Okay but its too big')

In [ ]:
sentiment_predict('This Tshirt fits me perfectly')